# Week 7 Notes

In [4]:
import pandas as pd
import numpy as np
import plotnine as plt
from plotnine import *
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from sklearn.model_selection import cross_val_score

warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
ames = pd.read_csv("C:/Users/hblin/Downloads/AmesHousing.csv")
X = ames.drop("SalePrice", axis=1)
y = ames["SalePrice"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 123)


In [41]:
# model 1
# Size and number of rooms
ct_dummies = ColumnTransformer(
  [("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])],
  remainder = "drop"
)

lr_pipeline = Pipeline(
  [("preprocessing", ct_dummies),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")


lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')

# Calculate the mean RMSE
mean_rmse = -scores.mean()  # Take the negative to convert back to positive RMSE
print(f"Mean RMSE: {mean_rmse}")

r2 = r2_score(y_test, y_pred)
print(f"r^2: {r2}")

cross_val_r2 = cross_val_score(lr_pipeline, X, y, cv=5, scoring='r2').mean()
print(f"Cross Val r^2: {cross_val_r2}")

RMSE: 50591.3232703246
Mean RMSE: 55806.32634926364
r^2: 0.5687214780092709
Cross Val r^2: 0.504208752508862


In [42]:
# model 2
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
  ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])],
  remainder = "drop"
)

lr_pipeline = Pipeline(
  [("preprocessing", ct_dummies),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")


lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')

# Calculate the mean RMSE
mean_rmse = -scores.mean()  # Take the negative to convert back to positive RMSE
print(f"Mean RMSE: {mean_rmse}")

r2 = r2_score(y_test, y_pred)
print(f"r^2: {r2}")

cross_val_r2 = cross_val_score(lr_pipeline, X, y, cv=5, scoring='r2').mean()
print(f"Cross Val r^2: {cross_val_r2}")

RMSE: 49047.62094866008
Mean RMSE: 54153.20862794976
r^2: 0.5946392954747666
Cross Val r^2: 0.5331485871994233


In [44]:
# First step: Dummy encoding and standardization
ct_dummies = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(drop="first", sparse_output=False), ["Bldg Type"]),
        ("standardize", StandardScaler(), ["Gr Liv Area"])
    ],
    remainder="drop"
).set_output(transform="pandas")

# Apply the preprocessing transformer first
X_train_transformed = ct_dummies.fit_transform(X_train)
X_test_transformed = ct_dummies.transform(X_test)

# Now you can access the encoded columns
encoded_columns = X_train_transformed.columns

# Define the interaction columns for the transformed (encoded) data
interaction_columns = [col for col in encoded_columns if "Bldg Type" in col]  # List of encoded "Bldg Type" columns

# Add the continuous feature column name
gr_liv_area_col = "standardize__Gr Liv Area"  # Verify this column name matches exactly

# Define interactions using transformed column names
# You can apply PolynomialFeatures on the relevant columns
ct_inter = ColumnTransformer(
    [
        ('interaction', PolynomialFeatures(interaction_only=True, include_bias=False),
         [gr_liv_area_col] + interaction_columns)
    ],
    remainder="drop"
).set_output(transform="pandas")

# Apply the interactions on top of the first transformation
X_train_interactions = ct_inter.fit_transform(X_train_transformed)
X_test_interactions = ct_inter.transform(X_test_transformed)

# Now, define the final pipeline for fitting
lr_pipeline = Pipeline([("linear_regression", LinearRegression())])

# Fit the model on transformed data
lr_pipeline.fit(X_train_interactions, y_train)
y_pred = lr_pipeline.predict(X_test_interactions)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

# Cross-validation on the transformed data
scores = cross_val_score(lr_pipeline, X_train_interactions, y_train, cv=5, scoring='neg_root_mean_squared_error')
mean_rmse = -scores.mean()
print(f"Mean RMSE: {mean_rmse}")

# Calculate r-squared for the test set
r2 = r2_score(y_test, y_pred)
print(f"r^2: {r2}")

# Cross-validation r-squared
cross_val_r2 = cross_val_score(lr_pipeline, X_train_interactions, y_train, cv=5, scoring='r2').mean()
print(f"Cross Val r^2: {cross_val_r2}")


RMSE: 48417.098629975684
Mean RMSE: 54751.69903028269
r^2: 0.6049943801381477
Cross Val r^2: 0.5353239424795158


In [39]:
# model 4
preprocessing = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("poly_size", Pipeline([
            ("standardize", StandardScaler()), 
            ("poly", PolynomialFeatures(degree=5, include_bias=False))
        ]), ["Gr Liv Area"]),
        ("poly_rooms", Pipeline([
            ("standardize", StandardScaler()), 
            ("poly", PolynomialFeatures(degree=5, include_bias=False))
        ]), ["TotRms AbvGrd"])
    ],
    remainder="drop"
).set_output(transform="pandas")

lr_pipeline = Pipeline(
  [("preprocessing", preprocessing),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

#lr_pipeline.named_steps["preprocessing"].get_feature_names_out()

lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')

# Calculate the mean RMSE
mean_rmse = -scores.mean()  # Take the negative to convert back to positive RMSE
print(f"Mean RMSE: {mean_rmse}")

r2 = r2_score(y_test, y_pred)
print(f"r^2: {r2}")

cross_val_r2 = cross_val_score(lr_pipeline, X, y, cv=5, scoring='r2').mean()
print(f"Cross Val r^2: {cross_val_r2}")

RMSE: 49092.35050568161
Mean RMSE: 55176.96594338035
r^2: 0.5938996113856301
Cross Val r^2: 0.5106643234404193


According to these models, model 3 seems to have performed the best.

With the cross validation, the RMSE's also agree that model 3 has performed the best. 

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
import numpy as np
import pandas as pd

# Define ColumnTransformer with separate PolynomialFeatures for each numeric feature
ct_poly = ColumnTransformer(
    transformers=[
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
        ("poly_gr_liv_area", PolynomialFeatures(include_bias=False), ["Gr Liv Area"]),
        ("poly_totrms_abvgrd", PolynomialFeatures(include_bias=False), ["TotRms AbvGrd"])
    ],
    remainder="drop"
)

# Define the pipeline with separate polynomial transformers
lr_pipeline_poly = Pipeline(
    steps=[
        ("preprocessing", ct_poly),
        ("linear_regression", LinearRegression())
    ]
).set_output(transform="pandas")

# Define a grid with separate degrees for each feature
degrees = {
    'preprocessing__poly_gr_liv_area__degree': np.arange(1, 11),
    'preprocessing__poly_totrms_abvgrd__degree': np.arange(1, 11)
}

# Perform GridSearchCV
gscv = GridSearchCV(lr_pipeline_poly, degrees, cv=5, scoring='r2')
gscv_fitted = gscv.fit(X, y)

# View the results
print(pd.DataFrame({
    "degrees_gr_liv_area": gscv.cv_results_['param_preprocessing__poly_gr_liv_area__degree'],
    "degrees_totrms_abvgrd": gscv.cv_results_['param_preprocessing__poly_totrms_abvgrd__degree'],
    "scores": gscv.cv_results_['mean_test_score']
}))

# Best parameters and score
print("Best parameters:", gscv.best_params_)
print("Best R-squared score:", gscv.best_score_)


    degrees_gr_liv_area  degrees_totrms_abvgrd     scores
0                     1                      1   0.532882
1                     1                      2   0.532383
2                     1                      3   0.535924
3                     1                      4   0.541529
4                     1                      5   0.541066
..                  ...                    ...        ...
95                   10                      6 -16.187618
96                   10                      7 -16.187618
97                   10                      8 -16.187618
98                   10                      9 -16.187618
99                   10                     10 -16.187618

[100 rows x 3 columns]
Best parameters: {'preprocessing__poly_gr_liv_area__degree': np.int64(3), 'preprocessing__poly_totrms_abvgrd__degree': np.int64(1)}
Best R-squared score: 0.5576406283340998


It seems that the model with size of degree 3 and rooms  of degree 1 perfomred the best. The downsides of this are that this could possibly take a long time in terms of computing power, and if we needed to do this quickly that would not be optimal. We could also limit the polynomial to a number of degrees that makes sense so we do not overfit the model or add too much complexity unnecessarily. 